# EMI mechanical model – part 3

In the first notebook you were introduced to the mechanical EMI model, and in the second one you explored how changing the contractility and matrix stiffness affected stress and strain patterns.

In this notebook we are going to explore the implications of doing both simultanously. We'll do a range of scaling values of scaling the calcium amplitude as well as a range of matrix stiffness values. This will give us $N^2$ different combinations of calcium and stiffness scaling parameters, which will be more expensive than before. We'll run the simulations first, then explore the parameter space using widgets.

We did the simulations for you – and saved the results in a file called <code>calcium_matrixstiffness_simulation_outputs.npy</code>, which contains a dictionary with relevant output values. We'll load these in and play with adjusting the parameters using widgets. In case you can't find the file, or in case you want to do some adjustments, the code for generating the file is included as <code>generate_calcium_matrixstiffness_simulation_outputs.py</code>.

## Changes in calcium amplitude and matrix stiffness

In [ ]:
import ipywidgets as widgets

fig, axes = plt.subplots(2, 2, figsize=(15, 6), sharex=True, sharey="col")

directions = ["fiber", "sheet", "normal"]
subdomains = [0, 1]         # extra-, intracellular subdomains
legends_strain = [r"$\overline{E_{ff}}$", r"$\overline{E_{ss}}$", r"$\overline{E_{nn}}$"]
legends_strain = [r"$\overline{\sigma_{ff}}$", r"$\overline{\sigma_{ss}}$", r"$\overline{\sigma_{nn}}$"]

output_values = np.load()

time = np.linspace(0, 500, 500) #output_values["time"]

@widgets.interact(ca_scale=(0.8, 1.22, 0.04), stiffness_scale=(0.8, 1.22, 0.04))
def f(ca_scale=1, stiffness_scale=1):
    print(ca_scale, stiffness_scale)

    for subdomain_id in subdomains:
        for direction in directions:
            
            strain_values = output_values["strain"][direction][subdomain_id]
            stress_values = output_values["stress"][direction][subdomain_id]
            
            axes[0][subdomain_id].plot(time, strain_values)
            axes[1][subdomain_id].plot(time, stress_values)

    axes[0][0].set_title(f"Extracellular subdomain ($\Omega_e$)")
    axes[1][0].set_title(f"Intracellular subdomain ($\Omega_i$)")

    axes[1][0].set_xlabel("Time (ms)")
    axes[1][1].set_xlabel("Time (ms)")
    axes[0][0].set_ylabel("Strain (-)")
    axes[1][0].set_ylabel("Stress (kPa)")

    axes[1][0].legend(legends_strain)
    axes[1][1].legend(legends_stress)
            
    plt.show()

## Running the simulations

This part will run all the simulations you need; it will take a while to run - so you can start the cell and go to take a break. Once you're back, you can use the widgets above to explore the impact across the predefined parameter range.

In [6]:
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict

import fenics as f
import emimechanicalmodel as emi_m

import rice_model_2008 as rice
from scipy.integrate import odeint


def calculate_force_from_the_rice_model(scaling_factor):
    p = rice.init_parameter_values(Ca_amplitude = scaling_factor_inner*1.45)
    force_ind = rice.monitor_indices("active")     

    init = rice.init_state_values()
    states = odeint(rice.rhs,init,time,(p,))

    force = []
    force_scaling_factor = 12
    
    for tn, sn in zip(time,states):
        m = rice.monitor(sn,tn,p)
        force.append(force_scaling_factor*m[force_ind])

    return np.array(force)


def run_forward_emi_simulation(emimodel, active_values):
    strain_values = defaultdict(dict)
    stress_values = defaultdict(dict)
    
    directions = ["fiber", "sheet", "normal"]
    subdomains = [0, 1]         # extra-, intracellular subdomains
    
    for direction in directions:
        for subdomain_id in subdomains:
            strain_values[direction][subdomain_id] = np.zeros_like(active_values)
            stress_values[direction][subdomain_id] = np.zeros_like(active_values)
        
    for step, a_str in enumerate(active_values):
        emimodel.update_active_fn(a_str)
        emimodel.solve()

        if step % 50 == 0:
            print("** time step: ", step)

        for subdomain_id in subdomains:
            strain_values["fiber"][subdomain_id][step] = \
                emimodel.evaluate_subdomain_strain_fibre_dir(subdomain_id)
            strain_values["sheet"][subdomain_id][step] = \
                emimodel.evaluate_subdomain_strain_sheet_dir(subdomain_id)
            strain_values["normal"][subdomain_id][step] = \
                emimodel.evaluate_subdomain_strain_normal_dir(subdomain_id)
        
            stress_values["fiber"][subdomain_id][step] = \
                emimodel.evaluate_subdomain_stress_fibre_dir(subdomain_id)
            stress_values["sheet"][subdomain_id][step] = \
                emimodel.evaluate_subdomain_stress_sheet_dir(subdomain_id)
            stress_values["normal"][subdomain_id][step] = \
                emimodel.evaluate_subdomain_stress_normal_dir(subdomain_id)
    
    return strain_values, stress_values


f.set_log_level(30)        # less verbatim output from the Newton solver compared to default

mesh_file = "meshes/tile_connected_10p0.h5"
mesh, volumes = emi_m.load_mesh(mesh_file)

scaling_factors = np.linspace(0.8, 1.2, 3)

print(scaling_factors)

time = np.linspace(0,500,500)

all_stress_values = defaultdict(dict)
all_strain_values = defaultdict(dict)

for step, scaling_factor in enumerate(scaling_factors):

    print(f"Running the simulation for outer step {1 + step} / {len(scaling_factors)}")
    
    material_parameters = {
        "C_e" : scaling_factor*0.5,
        }

    emimodel = emi_m.EMIModel(
            mesh,
            volumes,
            experiment="contraction",
            material_model="guccione",
            material_parameters=material_parameters,
            active_model="active_stress",
            compressibility_model="nearly_incompressible",
        )
    
    for step_inner, scaling_factor_inner in enumerate(scaling_factors):
        print(f"* Running the simulation for inner step {1 + step_inner} / {len(scaling_factors)}")

        active_values = calculate_force_from_the_rice_model(scaling_factor)
        strain_values, stress_values = run_forward_emi_simulation(emimodel, active_values)
    
        key_Ce = f"C_e scaling {scaling_factor}"
        key_Ca = f"Ca scaling {scaling_factor_inner}"
        
        all_stress_values[key_Ce][key_Ca] = stress_values
        all_strain_values[key_Ce][key_Ca] = strain_values
        
        emimodel.state.vector()[:] = 0       # make sure this is zero before starting a new simulation

all_output_values = {
        "strain" : all_strain_values,
        "stress" : all_stress_values,
             }

np.save("calcium_matrixstiffness_simulation_outputs.npy", all_output_values)

Mesh and subdomains loaded successfully.
Number of nodes: 335, number of elements: 1364
[0.8 1.  1.2]
Running the simulation for outer step 1 / 3
length=102.0, width=20.0, height=24.0
* Running the simulation for inner step 1 / 3
** time step:  0
** time step:  50
** time step:  100
** time step:  150
** time step:  200
** time step:  250
** time step:  300
** time step:  350
** time step:  400
** time step:  450
* Running the simulation for inner step 2 / 3
** time step:  0
** time step:  50
** time step:  100
** time step:  150
** time step:  200
** time step:  250
** time step:  300
** time step:  350
** time step:  400
** time step:  450
* Running the simulation for inner step 3 / 3
** time step:  0
** time step:  50
** time step:  100
** time step:  150
** time step:  200
** time step:  250
** time step:  300
** time step:  350
** time step:  400
** time step:  450
Running the simulation for outer step 2 / 3
length=102.0, width=20.0, height=24.0
* Running the simulation for inner 

We now have two dictionary with tracked strain and stress values, <code>all_stress_values</code> and <code>all_strain_values</code>.